<a href="https://colab.research.google.com/github/Anjasfedo/Learning-TensorFlow/blob/main/eat_tensorflow2_in_30_days/Chapter4_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4-5 AutoGraph and tf.Module

Introduce to constructing Autograph using `tf.Module`

## 1. Introduction to Autograph and `tf.Module`

The definition of `tf.Variable` should be avoided inside the decorator `@tf.function`.

However, it would seem to be a imperfect lealed package if define `tf.Variable` outside the functin, since the function has outside dependency.

One of the simple solutions is: defining a class and place the definition of `tf.Variable` inside the initial method, and leake the other methods/implementaion elsewhere.

After surprise is that TensorFlow providing a base class `tf.Module` to get the above neturally. What's more, it is supposed to be inherited for constructing child classes to manage variables and other `Module` conveniently. And the most important that it allows to save model through `tf.saved_model` and achieve cross-platfom deployment. What a surprise!

In fact, `tf.keras.models.Model`, `tf.keras.layers.Layer` are both inherited from `tf.Module`. They provides the management to the variables and the reffered sub-modules.

**We are able to develop arbitary learning model (not only neural network) and implement cross-platform deployment through the packaging provided by `tf.Module` and the low-level APIs in TensorFlow.**

## 2. Packaging Autograph Using `tf.Module`

Define a simple function.

In [6]:
import tensorflow as tf

x = tf.Variable(1.0, dtype=tf.float32)

# Use input_signature to limit the signature type of the input tensor with shape and dtype inside decorator tf.function
@tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])
def add_print(a):
    x.assign_add(a)
    tf.print(x)
    return(x)

In [7]:
add_print(tf.constant(3.0))
# add_print(tf.constant(3)) # Error: argument inconsistent with the tensor signature.

4


<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

Package using `tf.Module`.

In [8]:
class DemoModule(tf.Module):
  def __init__(self, init_value=tf.constant(0.0), name=None):
    super(DemoModule, self).__init__(name=name)
    with self.name_scope: # identical to: with tf.name_scope('demo_module')
      self.x = tf.Variable(init_value, dtype=tf.float32, trainable=True)

  @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])
  def addprint(self, a):
    with self.name_scope:
      self.x.assign_add(a)
      tf.print(self.x)

In [9]:
# Execute
demo = DemoModule(init_value=tf.constant(1.0))
result = demo.addprint(tf.constant(5.0))

6


In [10]:
# Browse all variable and trainable in the module
print(demo.variables)
print(demo.trainable_variables)

(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)
(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)


In [11]:
# Browse all sub-modules
demo.submodules

()

In [14]:
# Save the model using tf.saved_model and specify the method of cross-platfom deployment
tf.saved_model.save(demo, '/demo/1', signatures={'serving_default': demo.addprint})

ValueError: in user code:


    ValueError: Got a non-Tensor value None for key 'output_0' in the output of the function __inference_addprint_54 used to generate the SavedModel signature 'serving_default'. Outputs for functions used as signatures must be a single Tensor, a sequence of Tensors, or a dictionary from string to Tensor.


In [15]:
# Load the modle
demo2 = tf.saved_model.load("/demo/1")
demo2.addprint(tf.constant(5.0))

OSError: SavedModel file does not exist at: ../data/demo/1/{saved_model.pbtxt|saved_model.pb}

Check the graph in tensorboard, the module will be added t=wuth name `demo_module`, showing the hierarchy of the graph.